In [1]:
CLOUD_API_KEY = "******"

In [2]:
AIOS_GUID = "*****"

In [3]:
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

In [4]:
WML_CREDENTIALS = {
  "apikey": "**********",
  "iam_apikey_description": "Auto-generated for key *************",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "********",
  "instance_id": "************",
  "url": "https://us-south.ml.cloud.ibm.com"
}

This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

    **If you have previously configured OpenScale**, it will use your existing datamart, and not interfere with any models you are currently monitoring. **Do not update the cell below where DB_CREDENTIALS are filled out AND comment it out or do not run it. Instead, uncomment the cell for `DB_CREDENTIALS = NONE` AND RUN IT INSTEAD**.

**If you do not have a paid Cloud account or would prefer not to provision this paid service**, you may use the free internal PostgreSQL service with OpenScale. **Do not update the cell below where DB_CREDENTIALS are filled out AND comment it out or do not run it. Instead, uncomment the cell for `DB_CREDENTIALS = NONE` AND RUN IT INSTEAD**.

**To provision a new instance of Db2 Warehouse**:  
* Locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Db2 Warehouse credentials into the cell below.

**To provision a new instance of Databases for PostgreSQL**:
* Locate [Databases for PostgreSQL in the Cloud catalog](*https://cloud.ibm.com/catalog/services/databases-for-postgresql)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Databases for PostgreSQL credentials into the cell below.


IF you have previously configured OpenScale and are using an existing datamart OR would prefer not to provision the paid service, uncomment the cell below to set `DB_CREDENTIALS = NONE` AND do not run or comment out the cell where `DB_CREDENTIALS` are set.

In [5]:
DB_CREDENTIALS = None

Comment out the cell below if you have set `DB_CREDENTIALS = None`.

In [ ]:
/*
DB_CREDENTIALS = {
  "hostname": "***",
  "password": "***",
  "https_url": "***",
  "port": 50000,
  "ssldsn": "***",
  "host": "***",
  "jdbcurl": "***",
  "uri": "***",
  "db": "***",
  "dsn": "****",
  "username": "***",
  "ssljdbcurl": "****"
}
*/

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [6]:
KEEP_MY_INTERNAL_POSTGRES = True

# Configure OpenScale

In [32]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

## Create schema and datamart

In [33]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

'2.1.14'

#### Note: to delete an existing data_mart, uncomment the next cell and run it.

In [34]:
ai_client.data_mart.delete()




 Waiting for end of deleting data mart 




deleted

--------------------------------
 Successfully deleted data mart 
--------------------------------




### Set up datamart

In [35]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])

Setting up internal datamart


In [87]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'internal_database_pool': 'compose-psql',
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/47b84451ab70b94737518f7640a9ee42:b873054a-9264-48c4-bcfe-c462ac3b8cf8::',
 'status': {'state': 'active'}}

## Bind machine learning engines

In [37]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

ff6b6206-589d-436a-b80a-633d799499dc,WML instance,watson_machine_learning,2019-09-08T15:44:28.451Z


In [88]:
ai_client.data_mart.bindings.list_assets()

c4223e1e-8c5c-4115-904b-a0835059c5fb,9-8-TelcoChurn,2019-09-08T16:25:33.994Z,model,mllib-2.1,ff6b6206-589d-436a-b80a-633d799499dc,False
4d69cdfd-493c-43d4-9d80-0ae3d9ac4ceb,Spark German Risk Model - Final,2019-09-06T19:17:11.914Z,model,mllib-2.1,ff6b6206-589d-436a-b80a-633d799499dc,False
096eac1c-2f0b-4241-9c01-d00697642994,Spark ML Telco Customer Churn,2019-09-06T19:02:38.896Z,model,mllib-2.1,ff6b6206-589d-436a-b80a-633d799499dc,False


## Subscriptions

### Remove existing credit risk subscriptions

In [84]:
ai_client.data_mart.subscriptions.delete(WatsonMachineLearningAsset("c6903824-ae19-47e9-b466-ce0bf9b5dc1e"))

UnexpectedType: Unexpected type of 'subscription_uid', expected: '<class 'str'>', actual: '<class 'ibm_ai_openscale.engines.watson_machine_learning.asset.WatsonMachineLearningAsset'>'.

In [89]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [90]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["gender","SeniorCitizen","Partner","Dependents","tenure","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup", \
                       "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","MonthlyCharges","TotalCharges"],
    categorical_columns = ["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup", \
                           "DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod"]
), deployment_uids=deployment_uid,)

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [91]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

bb9b1186-bace-4cba-8205-2b6f2accff7e,9-8-TelcoChurn,model,ff6b6206-589d-436a-b80a-633d799499dc,2019-09-08T16:49:42.176Z
c195ee33-57bd-4d49-8594-401c481a6726,9-8-TelcoChurn,model,ff6b6206-589d-436a-b80a-633d799499dc,2019-09-08T15:45:29.922Z


In [92]:
subscription.get_details()

{'entity': {'asset': {'asset_id': 'c4223e1e-8c5c-4115-904b-a0835059c5fb',
   'asset_type': 'model',
   'created_at': '2019-09-08T16:25:33.994Z',
   'name': '9-8-TelcoChurn',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/published_models/c4223e1e-8c5c-4115-904b-a0835059c5fb'},
  'asset_properties': {'categorical_fields': ['gender',
    'Partner',
    'Dependents',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod'],
   'feature_fields': ['gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'tenure',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling

In [93]:
scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        scoring_endpoint = deployment['entity']['scoring_url']
        
print(scoring_endpoint)

7e41b473-ed3b-48c4-9213-70ec9626223c
https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/deployments/7e41b473-ed3b-48c4-9213-70ec9626223c/online


In [96]:
# Prepare the payload to be sent to the model
payload = {
    "fields": [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges"],
    "values": [
        ["Male", 0, "No", "No", 66, "Yes", "No", "Fiber optic", "Yes", "No", "Yes", "Yes", "Yes", "Yes", "Two year", "Yes", "Bank transfer (automatic)", 105.65, 6844.5 ]
    ]
}


In [97]:
# Send data to the model and print results
predictions = wml_client.deployments.score(scoring_endpoint, payload)
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "tenure",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
    "MonthlyCharges",
    "TotalCharges",
    "Churn",
    "gender_IX",
    "Partner_IX",
    "Dependents_IX",
    "PhoneService_IX",
    "MultipleLines_IX",
    "InternetService_IX",
    "OnlineSecurity_IX",
    "OnlineBackup_IX",
    "DeviceProtection_IX",
    "TechSupport_IX",
    "StreamingTV_IX",
    "StreamingMovies_IX",
    "Contract_IX",
    "PaperlessBilling_IX",
    "PaymentMethod_IX",
    "label",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      "Male",
      0,
      "No",
      "No",
      66,
      "Yes",
      "No",
      "Fiber optic",
      "Yes",
  

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [99]:
time.sleep(10)
subscription.quality_monitoring.enable(threshold=0.7, min_records=50)

### Feedback logging

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-44d72c3c-47a6-49ce-86a4-da434ac0c139',
    'IBM_API_KEY_ID': 'ftidKw54Prx3617F3JFUYz_MI3XwCcPjEU8i-R8F-EO2',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'telcocustomerchurn829-donotdelete-pr-nnvqez6nufw3tv',
    'FILE': 'telco_credit_feed.json'
}


In [106]:
scoring_data = {
"fields": ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
"values":
[
    ['Female', 0, 'Yes', 'No', 1, 'No', 'No phone service', 'DSL', 'No', 'Yes', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes', 'Electronic check', 29.85, 29.85, 'No'],
    ['Male', 0, 'No', 'No', 34, 'Yes', 'No', 'DSL', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'One year', 'No', 'Mailed check', 56.95, 1889.5, 'No'],
    ['Male', 0, 'No', 'No', 2, 'Yes', 'No', 'DSL', 'Yes', 'Yes', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes', 'Mailed check', 53.85, 108.15, 'Yes'],
    ['Male', 0, 'No', 'No', 45, 'No', 'No phone service', 'DSL', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'One year', 'No', 'Bank transfer (automatic)', 42.3, 1840.75, 'No'],
    ['Female', 0, 'No', 'No', 2, 'Yes', 'No', 'Fiber optic', 'No', 'No', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes', 'Electronic check', 70.7, 151.65, 'Yes'],
    ['Female', 0, 'No', 'No', 8, 'Yes', 'Yes', 'Fiber optic', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Month-to-month', 'Yes', 'Electronic check', 99.65, 820.5, 'Yes'],
    ['Male', 0, 'No', 'Yes', 22, 'Yes', 'Yes', 'Fiber optic', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Yes', 'Credit card (automatic)', 89.1, 1949.4, 'No'],
    ['Female', 0, 'No', 'No', 10, 'No', 'No phone service', 'DSL', 'Yes', 'No', 'No', 'No', 'No', 'No', 'Month-to-month', 'No', 'Mailed check', 29.75, 301.9, 'No'],
    ['Female', 0, 'Yes', 'No', 28, 'Yes', 'Yes', 'Fiber optic', 'No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Month-to-month', 'Yes', 'Electronic check', 104.8, 3046.05, 'Yes']
]
}

In [107]:
import random

#with open('german_credit_feed.json', 'r') as scoring_file:
 #   scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(scoring_endpoint, payload_scoring)
print(scoring_response)

{'fields': ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_IX', 'Partner_IX', 'Dependents_IX', 'PhoneService_IX', 'MultipleLines_IX', 'InternetService_IX', 'OnlineSecurity_IX', 'OnlineBackup_IX', 'DeviceProtection_IX', 'TechSupport_IX', 'StreamingTV_IX', 'StreamingMovies_IX', 'Contract_IX', 'PaperlessBilling_IX', 'PaymentMethod_IX', 'label', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['Male', 0, 'No', 'Yes', 22, 'Yes', 'Yes', 'Fiber optic', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Month-to-month', 'Yes', 'Credit card (automatic)', 89.1, 1949.4, 'No', 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, [18, [3, 5, 8, 11, 15, 16, 17], [1.0, 1.0, 1.0, 1.